In [ ]:
!pip install gen3_validator

In [ ]:
# import dictionaryutils as du
# dd = du.DataDictionary(root_dir="../examples/schema/yaml")
# # getting resolved schema
# dd.schema.get("lipidomics_file")

In [ ]:
from gen3_validator.resolve_schema import ResolveSchema

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()


In [ ]:
resolver.schema_resolved['demographic.yaml']

In [ ]:
# get the node order
resolver.node_order

In [ ]:
schema = resolver.schema_resolved['proteomics_file.yaml']
schema['properties']['data_file_properties.md5sum']

In [ ]:
# class to extract and store property, data type, and description

import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

schema = resolver.schema_resolved['unaligned_reads_file.yaml']

class NodeProps:
    def __init__(self, resolved_schema: dict):
        self.resolved_schema = resolved_schema
    
    def get_schema_name(self) -> str:
        schema_name = self.resolved_schema['title']
        return schema_name
    
    def get_prop_names(self) -> list:
        prop_names = list(self.resolved_schema['properties'].keys())
        return prop_names
    
    def get_data_file_prop_names(self) -> list:
        """
        Returns a list of property names under 'data_file_properties' if it exists,
        otherwise returns an empty list.
        """
        prop_names = self.get_prop_names()
        if "data_file_properties" in prop_names:
            return list(self.resolved_schema['properties']['data_file_properties'].keys())
        logger.warning(f"No data_file_properties found in {self.get_schema_name()}")
        return None
        

    def get_prop_info(self, prop_name: str) -> dict:
        prop_names  = self.get_prop_names()
        prop_data_file_names = self.get_data_file_prop_names()
        prop_info = None

        if prop_name in prop_names:
            prop_info = self.resolved_schema['properties'][prop_name]
        elif prop_data_file_names is not None and prop_name in prop_data_file_names:
            prop_info = self.resolved_schema['properties']['data_file_properties'][prop_name]
        else:
            logger.warning(f"Property '{prop_name}' not found in {self.get_schema_name()}")
            
        return prop_info

    def get_data_type(self, prop_name: str) -> str:
        prop_info = self.get_prop_info(prop_name)
        if prop_info is None:
            logger.warning(
                f"Property '{prop_name}' not found in {self.get_schema_name()}, could not pull type"
            )
            return None

        if "type" in prop_info and "pattern" in prop_info:
            prop_type = f"string | pattern = {prop_info['pattern']}"
        elif "type" in prop_info:
            prop_type = prop_info["type"]
        elif "enum" in prop_info:
            prop_type = "enum"
        else:
            logger.warning(
                f"Property '{prop_name}' has no 'type' or 'enum' key. "
                f"Could be an injected property, usually don't need "
                f"these in the template | prop_info = {prop_info}"
            )
            return None

        if not isinstance(prop_type, str):
            try:
                joined_types = ", ".join(prop_type)
                logger.warning(
                    f"Property type '{prop_type}' is not string, converting to string: {joined_types}"
                )
                return joined_types
            except TypeError:
                logger.warning(
                    f"Property type '{prop_type}' is not string and could not be joined."
                )
                return str(prop_type)

        return prop_type

    def get_description(self, prop_name: str) -> str:
        prop_info = self.get_prop_info(prop_name)
        if prop_info is None:
            logger.warning(f"""Property '{prop_name}' not found in {self.get_schema_name()}, could not pull description"""
            )
            return None
        prop_description = None
        
        if "description" in prop_info:
            prop_description = prop_info['description']
        if "term" in prop_info:
            prop_description = prop_info['term']["description"]
        
        if prop_description is None:
            logger.warning(f"""Property '{prop_name}' has no description key. 
                Could be an injected property, usually don't need these in the
                template | prop_info = {prop_info}"""
            )
        
        return prop_description
        
# usage
nodeprops = NodeProps(schema)
# nodeprops.get_description("md5sum")
nodeprops.get_data_file_prop_names()

In [ ]:
from gen3_metadata_templates.props import NodeProps
schema = resolver.schema_resolved['unaligned_reads_file.yaml']
nodeprops = NodeProps(schema)
nodeprops.get_data_type('md5sum')

In [ ]:
nodeprops.get_data_file_prop_names()

In [ ]:
prop_names = nodeprops.get_prop_names()
data_file_prop_names = nodeprops.get_data_file_prop_names()
if data_file_prop_names is not None:
    prop_names = prop_names.append(data_file_prop_names)
prop_names

In [ ]:
descriptions = {}
for prop_name in prop_names:
    descriptions[prop_name] = nodeprops.get_description(prop_name)
descriptions

In [ ]:
types = {}
for prop_name in prop_names:
    types[prop_name] = nodeprops.get_data_type(prop_name)
types

In [ ]:
import pytest
from gen3_validator.resolve_schema import ResolveSchema

@pytest.fixture
def fixture_res_schema():
    resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
    resolver.resolve_schema()
    return resolver.schema_resolved['unaligned_reads_file.yaml']

def test_init_NodeProps(fixture_res_schema):
    nodeprops = NodeProps(fixture_res_schema)
    assert fixture_res_schema == nodeprops.resolved_schema

In [ ]:
t = ", ".join({"one": "test", "two": "test"})
t

In [1]:
from gen3_validator.resolve_schema import ResolveSchema
from gen3_metadata_templates.props import NodeProps

resolver = ResolveSchema(schema_path="../examples/schema/json/acdc_schema.json")
resolver.resolve_schema()

node_props = NodeProps(resolver.schema_resolved['unaligned_reads_file.yaml'])


node_props.get_schema_name()

# return prop names for the schema
prop_names = node_props.get_prop_names()
print(prop_names)

# return the data types for the properties
types = {}
for prop_name in prop_names:
    types[prop_name] = node_props.get_data_type(prop_name)
print(types)

# return the description for the properties
descriptions = {}
for prop_name in prop_names:
    descriptions[prop_name] = node_props.get_description(prop_name)
print(descriptions)

INFO:gen3_validator.resolve_schema:Initializing ResolveSchema with schema path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Starting schema resolution process.
INFO:gen3_validator.resolve_schema:Reading JSON file from path: ../examples/schema/json/acdc_schema.json
INFO:gen3_validator.resolve_schema:Successfully read JSON schema.
INFO:gen3_validator.resolve_schema:Retrieving node names from schema.
INFO:gen3_validator.resolve_schema:Retrieved 34 nodes from schema.
INFO:gen3_validator.resolve_schema:Retrieving all node pairs, excluding specified nodes.
INFO:gen3_validator.resolve_schema:Finding upstream and downstream nodes for: demographic.yaml
INFO:gen3_validator.resolve_schema:Retrieving links and ID for node: demographic.yaml
INFO:gen3_validator.resolve_schema:Finding upstream and downstream nodes for: project.yaml
INFO:gen3_validator.resolve_schema:Retrieving links and ID for node: project.yaml
INFO:gen3_validator.resolve_schema:Finding upstream and d

['type', 'id', 'submitter_id', 'state', 'project_id', 'created_datetime', 'updated_datetime', 'file_name', 'file_size', 'file_format', 'md5sum', 'object_id', 'file_state', 'error_type', 'ga4gh_drs_uri', 'genomics_assay', 'core_metadata_collections', 'data_category', 'data_format', 'data_type', 'run_id']
{'type': 'string', 'id': 'string | pattern = ^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$', 'submitter_id': 'string', 'state': None, 'project_id': 'string', 'created_datetime': None, 'updated_datetime': None, 'file_name': 'string', 'file_size': 'integer', 'file_format': 'string', 'md5sum': 'string | pattern = ^[a-f0-9]{32}$', 'object_id': 'string', 'file_state': 'enum', 'error_type': 'enum', 'ga4gh_drs_uri': 'string', 'genomics_assay': None, 'core_metadata_collections': None, 'data_category': 'enum', 'data_format': 'enum', 'data_type': 'enum', 'run_id': 'string'}
{'type': None, 'id': 'A 128-bit identifier. Depending on the mechanism used to generate it, 